<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/H_spell_check%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_C_inc.csv	   df_K_inc.csv     spell_check
'Colab Notebooks'	'# df_G_inc.csv'   df_P_inc.csv
 C_ori_model.pt		 df_G_inc.csv	  'My Drive'
'답안 작성용 파일.csv'	 df_J_inc.csv	   Q_inc_model.pt


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [4]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [5]:
df=df[(df['digit_1'] =='H')] #조건에 맞는 데이터 출력

In [6]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000018,H,52,529,"선박,화물",운송화물,검수
1,id_0000043,H,49,493,개별화물자동차로,고객의 요청으로,화물운송서비스
2,id_0000061,H,49,493,용달화물자동차로,고객의 요청으로,화물운송서비스
3,id_0000063,H,52,529,시청에서,주민의 편의를 위해,주차장운용
4,id_0000103,H,49,493,개별화물자동차로,고객의 요청으로,화물운송서비스
...,...,...,...,...,...,...,...
98033,id_0999948,H,49,494,사업장에서,도시내에서소화물을,물류배달제공
98034,id_0999961,H,49,493,용달화물자동차로,고객의 요청으로,화물운송서비스
98035,id_0999962,H,49,492,개인택시로,일반인을 대상으로,승객운송서비스
98036,id_0999982,H,49,493,일반화물자동차로,고객의 요청으로,화물운송서비스


In [7]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000018,H,52,529,"선박,화물 운송화물 검수"
1,id_0000043,H,49,493,개별화물자동차로 고객의 요청으로 화물운송서비스
2,id_0000061,H,49,493,용달화물자동차로 고객의 요청으로 화물운송서비스
3,id_0000063,H,52,529,시청에서 주민의 편의를 위해 주차장운용
4,id_0000103,H,49,493,개별화물자동차로 고객의 요청으로 화물운송서비스
...,...,...,...,...,...
98033,id_0999948,H,49,494,사업장에서 도시내에서소화물을 물류배달제공
98034,id_0999961,H,49,493,용달화물자동차로 고객의 요청으로 화물운송서비스
98035,id_0999962,H,49,492,개인택시로 일반인을 대상으로 승객운송서비스
98036,id_0999982,H,49,493,일반화물자동차로 고객의 요청으로 화물운송서비스


In [8]:
df['digit_3'].value_counts()

493    49115
492    39176
529     5605
494     2134
521     1591
501      281
491       58
502       34
511       32
512        9
495        3
Name: digit_3, dtype: int64

## 맞춤법 교정

In [9]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-7tfrdxjg
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-7tfrdxjg
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=885fe5d886c92d6a33c9d0e1a86138a9d40dc0832c2a3065e5b5ade523367347
  Stored in directory: /tmp/pip-ephem-wheel-cache-o511c09l/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [10]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [11]:
df.to_csv('df_H_spellcheck.csv', index=False, encoding='utf-8-sig')